# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

**Используемые библиотеки:**

In [ ]:
import time
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from catboost import CatBoostRegressor
import xgboost as xg
import lightgbm as lgb

from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

**Глобальные переменные:**

In [ ]:
state = np.random.RandomState(12345)

## Подготовка данных

### Загрузка данных

In [ ]:
try:
    df = pd.read_csv('autos.csv')
except:
    df = pd.read_csv('/datasets/autos.csv')

In [ ]:
df.head()

In [ ]:
df.info()

#### Вывод:
Данные загружены, проведен первичный осмотр. Данные загрузились без каких-либо проблем!

### Работа с пропусками 

In [ ]:
df.isna().sum()

Как видим, есть 5 признаков, в которых присутствуют пропуски. Пройдемся по каждому из них:

VehicleType -- этот признак невозможно избавить от пропусков, не прибегая к источникам из вне, так как машина одной модели может иметь разный тип кузова, например `volkswagen golf`, поэтому придется заполнить пропуски заглушками

Gearbox -- этот признак можно попытаться заполнить основываясь на на том, что машина одних и тех же годов, марки и модели скорее всего имеет схожую коробку передач

Model -- этот признак невозможно восстановить, так как у одной марки довольно большой модельный ряд, тем более нельзя опираться на косвенные признаки, так как автопроизводители устанавливают одни и те же детали в разные модели.

FuelType -- этот признак тоже можно попытаться заполнить, так как у машин одной марки, модели, мощности будут скорее всего схожие типы топлива

Repaired -- этот признак невозможно восполнить основываясь на датасете, так как каждый автомобиль независим, поэтому марка и модель не будут иметь влияние на этот признак.

Сначала найдем все строки, в которых есть пропуски во всех пяти признаках, такие строки не предоставляют из себя ценности, поэтому можно их удалить.

In [ ]:
nans_indexes = df.loc[df['Repaired'].isna() & 
                      df['VehicleType'].isna() & 
                      df['Model'].isna() &
                      df['Gearbox'].isna() &
                      df['FuelType'].isna()].index
nans_indexes

In [ ]:
df = df.drop(index=nans_indexes)

На мой взгляд, все признаки, относящиеся к:
- дате регистрации
- описанию продавца

не имеют влияния на цену машины

In [ ]:
df = df.drop(['DateCrawled', 'RegistrationMonth', 'PostalCode', 'LastSeen', 'NumberOfPictures', 'DateCreated'], axis=1)

Далее надо посмотреть есть ли объекты с нулевой ценой

In [ ]:
df.loc[df.Price < 100]

Таких объявлений много, их надо убрать, так как это сильно повлияет на качество модели

In [ ]:
df = df.loc[df.Price > 100]
df.head(3)

Заполним заглушками следущие признаки:

In [ ]:
df[['VehicleType', 'Model', 'Repaired']] = df[['VehicleType', 'Model', 'Repaired']].fillna('unknown')
df['FuelType'] = df['FuelType'].fillna('other')

Далее создадим функцию, которая будет заполнять пропуски в `Gearbox`, основываясь на идее, что у машин одинакового года, модели, бренда и мощности схожие типы коробок передач.

In [ ]:
def fill_gaps(row):
    
    if row['Gearbox'] not in ['auto', 'manual']:
        try:
            row['Gearbox'] = df.loc[(df['Brand'] == row['Brand']) & 
                                    (df['Model'] == row['Model']) & 
                                    (df['RegistrationYear'] == row['RegistrationYear']) & 
                                    (df['Power'] == row['Power']), 'Gearbox'].mode()[0]
        except:
            row['Gearbox'] = 'unknown'
            
    return row

In [ ]:
%%time

new_df = df.apply(fill_gaps, axis=1)

In [ ]:
new_df.isna().sum()

Оставшиеся признаки нельзя восстановить по данным датасета, поэтому есть только два выхода: удалить эти строки или заполнить пропуски заглушками. Так как в сумме пропуски составляют 24% от общего объема всего датасета, удаление этих строк сильно скажется на качестве датасета.

#### Вывод
Пропуски побеждены!

### Аномалии

Аномалии могут встретиться только в численных признках, то есть в `Price`, `Power`, `Kilometer`.

In [ ]:
sns.boxplot(new_df[['Price', 'Power', 'Kilometer']])
plt.xlabel("Признаки")
plt.ylabel("Разброс")
plt.title('Ящик с усами для всех количественных признаков');

In [ ]:
sns.boxplot(new_df[['Price', 'Power', 'Kilometer']])
plt.xlabel("Признаки")
plt.ylabel("Разброс")
plt.title('Ящик с усами для всех количественных признаков')
plt.ylim((0, 7000));

Из двух ящиков с усами можно сделать следующий вывод: 

Во-первых, мы видим огромное количество выбросов в признке `Power`, нужно посмотреть на общее количество таких выбросов и решить удалять такие объекты или все таки исправлять

Во-вторых, на мой взгляд, в признаке `Kilometer` не стоит убирать аномальные значения пробега, так как их не сособо много, и они не сильно портят общую картину.


In [ ]:
new_df['Power'].describe()

Можно было бы стандартизацию провести, но STD огромный из-за выбросов, поэтому получится фигня, на мой взгляд порог в 500 лошадиных сил адекватен, выбросов, которые привосходят этот порог около 500 штук, поэтому резоннее просто удалить такие объекты, чем восстанавливать верные значения для каждого из них.

Также стоит удалить объекты, которые имеют много пропусков и мощность аномально низкая

In [ ]:
new_df = new_df.loc[(new_df['Power'] > 40) & 
                   (new_df['VehicleType'] != 'unknown') & 
                   (new_df['Repaired'] != 'unknown') & 
                   (new_df['Model'] != 'unknown')]

In [ ]:
new_df = new_df.loc[new_df['Power'] < 500]

In [ ]:
power_pivot_table = round(new_df.pivot_table(index='Brand', values='Power', aggfunc='median'))
power_pivot_table.head(3)

In [ ]:
def anomalies(row):
    if row['Power'] < 10:
        row['Power'] == power_pivot_table['Power'][row['Brand']]
    return row

In [ ]:
%%time

new_df = new_df.apply(anomalies, axis=1)

Теперь посмотрим на аномалии в годе регистрации автомобиля

In [ ]:
new_df['RegistrationYear'].describe()

In [ ]:
new_df = new_df.loc[new_df['RegistrationYear'] > 1980]

#### Вывод:
Почистили датасет от аномалий в признаке `Power` и `RegistrationYear`

## Обучение моделей

### Подготовка выборок для обучения

Деление датасета на выборки:

In [ ]:
target = new_df['Price']
features = new_df.drop(['Price'], axis=1)

features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                                            test_size=0.25, random_state=state)
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, 
                                                                              test_size=0.25, random_state=state)

Стандартизация данных:

In [ ]:
numeric = ['RegistrationYear', 'Power', 'Kilometer']

scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

### Обучение моделей

Перед началом напишем функцию, которая будет высчитывать среднее время предсказания для n-го числа объектов

In [ ]:
def check_time(model, data, len_sample, count_of_preds):
    times = []
    for i in range(count_of_preds):
        start = time.time()
        predict = model.predict(data.sample(len_sample))
        stop = time.time()
        times.append(stop - start)
    return round(np.array(times).mean(), 5)

**RMSE**

In [ ]:
def RMSE(target, predict):
    return np.sqrt(mean_squared_error(target, predict))

rmse = make_scorer(RMSE, greater_is_better = False)

**CatBoostRegressor**

Запишем категориальные переменные

In [ ]:
cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']

Лучшие гиперпараметры мы будем искать, используя GridSearchCV

In [ ]:
grid_params = {'learning_rate': [0.1, 0.2, 0.4],
        'depth': [4, 6, 8],
        'iterations': [100, 50, 200]}

In [ ]:
%%time

model = CatBoostRegressor()
grid = GridSearchCV(model, grid_params, scoring=rmse, n_jobs=-1, verbose=50)
grid.fit(features_train, target_train, cat_features=cat_features)

Теперь посмотрим на среднее время обучения, предсказания и RMSE

In [ ]:
grid.cv_results_

Теперь посмотрим на метрики, важные для заказчика

In [ ]:
print('Результаты для CatBoostRegressor')
print('Среднее время обучения:', round(grid.cv_results_['mean_fit_time'].mean(), 2))
print("Среднее время предсказания:", round(grid.cv_results_['mean_score_time'].mean(), 3))
print("RMSE:", round(np.abs(grid.cv_results_['mean_test_score']).mean(), 1))

Теперь, используя подобранные параметры, обучим модель и получим метрику RMSE

In [ ]:
%%time

model_catboost = CatBoostRegressor(verbose=100, **grid.best_params_, loss_function='RMSE')
model_catboost.fit(features_train, target_train, cat_features=cat_features)
predict_valid = model_catboost.predict(features_valid)

In [ ]:
print(f'RMSE для CatBoostRegressor: {round(RMSE(predict_valid, target_valid), 1)}')

**LightGBM**

In [ ]:
ohe_encoder = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore')

train_temp = ohe_encoder.fit_transform(features_train[cat_features])
features_train[ohe_encoder.get_feature_names_out()] = train_temp
features_train.drop(cat_features, axis=1, inplace=True)

valid_temp = ohe_encoder.transform(features_valid[cat_features])
features_valid[ohe_encoder.get_feature_names_out()] = valid_temp
features_valid.drop(cat_features, axis=1, inplace=True)

test_temp = ohe_encoder.transform(features_test[cat_features])
features_test[ohe_encoder.get_feature_names_out()] = test_temp
features_test.drop(cat_features, axis=1, inplace=True)

In [ ]:
grid_params = {
    'objective': ['regression'],
    'metric': ['RMSE'],
    'learning_rate': [0.005, 0.01],
    "max_depth": [4, 8, 12, 16],
    "num_leaves": [64, 128, 256],
    'n_estimations': [500, 1000, 1500]
}

In [ ]:
lgb_regressor = lgb.LGBMRegressor(learning_rate=0.01)
grid = GridSearchCV(lgb_regressor, param_grid=grid_params, cv=5, verbose=100)
grid.fit(features_train, target_train)
grid.best_params_

In [ ]:
grid.cv_results_

In [ ]:
print('Результаты для LGBMRegressor')
print('Среднее время обучения:', round(grid.cv_results_['mean_fit_time'].mean(), 2))
print("Среднее время предсказания:", round(grid.cv_results_['mean_score_time'].mean(), 3))
print("RMSE:", round(np.abs(grid.cv_results_['mean_test_score']).mean(), 1))

In [ ]:
train_df = lgb.Dataset(features_train, label=target_train)
booster = lgb.train({
 'max_depth': 16,
 'metric': 'RMSE',
 'n_estimations': 500,
 'num_leaves': 256,
 'objective': 'regression'}, train_set=train_df)

In [ ]:
predict_valid = booster.predict(features_valid)
print(f'RMSE для LGBMRegressor: {round(RMSE(predict_valid, target_valid), 2)}')

**LinearRegression**

In [ ]:
model_lin_reg = LinearRegression(n_jobs=-1)
start_0 = time.time()
model_lin_reg.fit(features_train, target_train)
stop_0 = time.time()
start_1 = time.time()
predict_valid = model_lin_reg.predict(features_valid)
stop_1 = time.time()
print(f'RMSE для LinearRegression: {round(RMSE(predict_valid, target_valid), 2)}')

In [ ]:
print('Результаты для CatBoostRegressor')
print('Среднее время обучения:', round(abs(start_0 - stop_0), 2))
print("Среднее время предсказания:", round(abs(start_1 - stop_1), 3))
print("RMSE:", round(RMSE(predict_valid, target_valid), 1))

#### Вывод:
Как видим, модели, использующие градиентный бустинг намного лучше справляются с задачей регрессии.

### Анализ моделей

Очевидно, что линейная регрессия точно не является лучшей моделью по сравнению с моделями, использующими градиентный бустинг, поэтому выбирать приходится из CatBoost и LightGBM.

Сравнивая RMSE, можно придти к выводу, что LightGBM лучше справляется с точностью предсказания цены автомобиля.

Что касается скорости обученя, то тут LightGBM опять же лучше справляется 1 секунда против 32 у CatBoost.

И, наконец, скорость предсказания у LightGBM тоже лучше, чем у Catboost.

На такой результат могло повлиять, что в подборе гиперпараметров для LightGBM было 72 комбинации, против 27 у Catboost, и возможно из-за этого LightGBM имеет лучше подогнанные гиперпараметры, а также может сказаться, что для Catboost признаки не были закодированы. 

Чтобы проверить последнее замечание я проведу еще один GridSearch для Catboost но уже с закодированными признаками.

In [ ]:
grid_params = {'learning_rate': [0.1, 0.2, 0.4],
        'depth': [4, 6, 8],
        'iterations': [100, 50, 200]}

In [ ]:
%%time

model = CatBoostRegressor()
grid = GridSearchCV(model, grid_params, scoring=rmse, n_jobs=-1, verbose=50)
grid.fit(features_train, target_train)

In [ ]:
print('Результаты для LGBMRegressor')
print('Среднее время обучения:', round(grid.cv_results_['mean_fit_time'].mean(), 2))
print("Среднее время предсказания:", round(grid.cv_results_['mean_score_time'].mean(), 3))
print("RMSE:", round(np.abs(grid.cv_results_['mean_test_score']).mean(), 1))

Как мы видим, кодирование признаков положительно повлияла на время обучения и время предсказания. Однако ситуация не поменялась, LightGBM все еще лучше.

Протестируем модель LGBMRergressor на тестовой выборке:

In [ ]:
predict_test = booster.predict(features_test)
print(f'RMSE для LGBMRegressor: {round(RMSE(predict_test, target_test), 2)}')

Как видим, метрика RMSE ниже 2500, а значит модель рабоет хорошо!

#### Вывод:
По итогам просмотра метрик, важдых для заказчика была выбрана модель LGBMRegressor, так как она справляется с задачей быстрее и точнее, чем другие аналоги, использующие градиентный бустинг.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнена загрузка и подготовка данных
- [x]  Выполнено обучение моделей
- [x]  Есть анализ скорости работы и качества моделей